In [1]:
"""
SHAP 분석 - XGBoost 모델의 Feature 기여도 심층 분석 (전 포지션 자동화)
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import shap
import platform
import os
import glob
import joblib
from datetime import datetime
from pathlib import Path


# ==========================================
# 0. 설정
# ==========================================
def set_korean_font():
    system_name = platform.system()
    if system_name == 'Darwin':
        plt.rcParams['font.family'] = 'AppleGothic'
    elif system_name == 'Windows':
        plt.rcParams['font.family'] = 'Malgun Gothic'
    else:
        plt.rcParams['font.family'] = 'NanumGothic'
    plt.rcParams['axes.unicode_minus'] = False


# 경로 설정
DATA_PATH = "data/final_features.csv"
MODEL_DIR = "models"
OUTPUT_BASE_DIR = Path("output/shap_analysis")

# 분석 대상 포지션
TARGET_POSITIONS = ['TOP', 'JUNGLE', 'MIDDLE', 'BOT_DUO']

# [중요] 학습 코드와 동일한 20개 Feature (minute, level 제외)
FEATURE_COLS = [
    # 기본
    'xp', 'minionsKilled', 'jungleMinionsKilled',
    'totalDamageDoneToChampions', 'totalDamageTaken', 'timeEnemySpentControlled',
    'kills', 'deaths', 'assists', 'soloKills', 'turretPlates', 'wardsPlaced', 'objectiveParticipation',
    # 차이
    'xpDiff', 'csDiff', 'damageDiff', 'killsDiff',
    # 비율
    'KDA', 'killParticipation', 'damageShare'
]

TARGET_COL = 'totalGold'


# ==========================================
# 1. 데이터 및 모델 로드
# ==========================================
def find_latest_model(position):
    """해당 포지션의 최신 모델 파일 찾기"""
    search_pattern = os.path.join(MODEL_DIR, f"xgb_gold_predictor_{position}_*.pkl")
    files = glob.glob(search_pattern)
    if not files:
        return None
    return max(files, key=os.path.getctime)


def load_data_and_model(position):
    """포지션별 데이터 필터링 및 모델 로드"""
    print(f"\n📂 [{position}] 데이터 및 모델 로딩 중...")

    # 1. 데이터 로드
    if not os.path.exists(DATA_PATH):
        print(f"❌ 데이터 파일 없음: {DATA_PATH}")
        return None, None

    df = pd.read_csv(DATA_PATH)
    df = df[df['position'] == position].reset_index(drop=True)
    df = df[df['minute'] > 0]  # 0분 제외

    if df.empty:
        print(f"⚠️ {position} 데이터가 없습니다.")
        return None, None

    print(f"   데이터 크기: {len(df):,}행")

    # 2. 모델 로드
    model_path = find_latest_model(position)
    if not model_path:
        print(f"⚠️ {position} 모델 파일을 찾을 수 없습니다. (models/ 폴더 확인)")
        return None, None

    try:
        model = joblib.load(model_path)
        print(f"   ✅ 모델 로드 완료: {os.path.basename(model_path)}")
    except Exception as e:
        print(f"❌ 모델 로드 실패: {e}")
        return None, None

    return df, model


# ==========================================
# 2. SHAP 분석
# ==========================================
def run_shap_analysis(model, df, sample_size=5000):
    """SHAP 분석 실행"""
    print(f"   SHAP 값 계산 시작 (샘플: {sample_size:,}행)...")

    # 샘플링
    if len(df) > sample_size:
        df_sample = df.sample(sample_size, random_state=42)
    else:
        df_sample = df

    # 학습에 사용된 Feature만 추출
    X_sample = df_sample[FEATURE_COLS]

    # SHAP Explainer 생성
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_sample)

    print("   ✅ SHAP 계산 완료")
    return X_sample, shap_values, explainer, df_sample


# ==========================================
# 3. SHAP 시각화
# ==========================================
def plot_shap_summary(X_sample, shap_values, save_path):
    plt.figure(figsize=(12, 10))
    shap.summary_plot(shap_values, X_sample, show=False, max_display=20)
    plt.title("SHAP Summary Plot - Feature Importance", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()


def plot_shap_bar(X_sample, shap_values, save_path):
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X_sample, plot_type="bar", show=False, max_display=20)
    plt.title("SHAP Feature Importance (Mean |SHAP|)", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()


def plot_shap_dependence(X_sample, shap_values, feature, save_path):
    # 해당 Feature가 존재하는지 확인
    if feature not in X_sample.columns:
        return

    plt.figure(figsize=(10, 6))
    shap.dependence_plot(feature, shap_values, X_sample, show=False)
    plt.title(f"SHAP Dependence Plot: {feature}", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()


def plot_shap_waterfall_sample(explainer, X_sample, shap_values, idx, save_path):
    plt.figure(figsize=(12, 8))
    shap.plots._waterfall.waterfall_legacy(
        explainer.expected_value,
        shap_values[idx],
        feature_names=X_sample.columns.tolist(),
        max_display=15,
        show=False
    )
    plt.title(f"SHAP Waterfall - Sample #{idx}", fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()


def analyze_shap_by_time(df_sample, shap_values, save_path):
    """시간대별 SHAP 중요도 분석 (데이터프레임에 minute 정보가 있어야 함)"""
    if 'minute' not in df_sample.columns:
        return pd.DataFrame()

    print("   시간대별 SHAP 분석 중...")

    # SHAP 값을 DataFrame으로 변환
    shap_df = pd.DataFrame(np.abs(shap_values), columns=FEATURE_COLS)
    shap_df['minute'] = df_sample['minute'].values

    time_periods = {
        'Early (1-10m)': (1, 10),
        'Mid (11-20m)': (11, 20),
        'Late (21m+)': (21, 100)
    }

    results = {}
    for period_name, (start, end) in time_periods.items():
        period_mask = (shap_df['minute'] >= start) & (shap_df['minute'] <= end)
        if period_mask.sum() > 0:
            # 해당 구간의 평균 절대 SHAP 값
            mean_shap = shap_df.loc[period_mask, FEATURE_COLS].mean()
            results[period_name] = mean_shap

    if not results:
        return pd.DataFrame()

    results_df = pd.DataFrame(results)

    # 시각화
    fig, ax = plt.subplots(figsize=(14, 10))
    results_df.plot(kind='barh', ax=ax, width=0.8)

    ax.set_xlabel('Mean |SHAP Value|', fontsize=12)
    ax.set_title('Time-based Feature Importance Change', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()

    return results_df


def print_shap_summary(X_sample, shap_values):
    """텍스트 요약 출력"""
    mean_abs_shap = np.abs(shap_values).mean(axis=0)
    shap_df = pd.DataFrame({
        'Feature': X_sample.columns,
        'Mean |SHAP|': mean_abs_shap
    }).sort_values('Mean |SHAP|', ascending=False)

    # 상위 5개만 출력
    print(f"   🔥 Top 5 Features: {shap_df.head(5)['Feature'].tolist()}")
    return shap_df


# ==========================================
# 4. 메인 실행
# ==========================================
def main():
    set_korean_font()
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    for pos in TARGET_POSITIONS:
        # 1. 로드
        df, model = load_data_and_model(pos)
        if df is None or model is None:
            continue

        # 2. 폴더 생성
        output_dir = OUTPUT_BASE_DIR / pos
        output_dir.mkdir(parents=True, exist_ok=True)

        # 3. SHAP 분석 실행
        # 샘플 사이즈는 속도를 위해 5000개 정도로 설정 (필요시 조정)
        X_sample, shap_values, explainer, df_sample = run_shap_analysis(model, df, sample_size=5000)

        # 4. 요약 저장
        shap_df = print_shap_summary(X_sample, shap_values)
        shap_df.to_csv(output_dir / f"shap_importance.csv", index=False)

        # 5. 시각화 저장
        plot_shap_summary(X_sample, shap_values, output_dir / "shap_summary.png")
        plot_shap_bar(X_sample, shap_values, output_dir / "shap_bar.png")

        # 주요 Feature 의존성 그래프 (상위 3개)
        top_features = shap_df.head(3)['Feature'].tolist()
        for feat in top_features:
            plot_shap_dependence(X_sample, shap_values, feat, output_dir / f"shap_dependence_{feat}.png")

        # Waterfall Plot (랜덤 샘플 1개)
        plot_shap_waterfall_sample(explainer, X_sample, shap_values, 0, output_dir / "shap_waterfall_sample.png")

        # 시간대별 분석
        # df_sample에는 minute 정보가 포함되어 있으므로 이를 활용
        analyze_shap_by_time(df_sample, shap_values, output_dir / "shap_by_time.png")

        print(f"   ✅ {pos} 분석 완료 -> {output_dir}")

    print("\n" + "=" * 60)
    print(f"🎉 모든 포지션 SHAP 분석 완료! ({OUTPUT_BASE_DIR})")
    print("=" * 60)


if __name__ == "__main__":
    main()


📂 [TOP] 데이터 및 모델 로딩 중...
   데이터 크기: 2,177,986행
   ✅ 모델 로드 완료: xgb_gold_predictor_TOP_20251207_171358.pkl
   SHAP 값 계산 시작 (샘플: 5,000행)...
   ✅ SHAP 계산 완료
   🔥 Top 5 Features: ['xp', 'minionsKilled', 'kills', 'totalDamageDoneToChampions', 'turretPlates']


/opt/homebrew/anaconda3/lib/python3.13/site-packages/shap/plots/_waterfall.py:638: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) AppleGothic.
  text_bbox = txt_obj.get_window_extent(renderer=renderer)
/var/folders/3r/fkfjb6j92yl5dmn63k5ggz4m0000gn/T/ipykernel_88531/4239639834.py:171: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) AppleGothic.
  plt.savefig(save_path, dpi=150, bbox_inches='tight')


   시간대별 SHAP 분석 중...
   ✅ TOP 분석 완료 -> output/shap_analysis/TOP

📂 [JUNGLE] 데이터 및 모델 로딩 중...
   데이터 크기: 2,177,986행
   ✅ 모델 로드 완료: xgb_gold_predictor_JUNGLE_20251207_171358.pkl
   SHAP 값 계산 시작 (샘플: 5,000행)...
   ✅ SHAP 계산 완료
   🔥 Top 5 Features: ['xp', 'totalDamageDoneToChampions', 'jungleMinionsKilled', 'kills', 'minionsKilled']


/opt/homebrew/anaconda3/lib/python3.13/site-packages/shap/plots/_waterfall.py:638: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) AppleGothic.
  text_bbox = txt_obj.get_window_extent(renderer=renderer)
/var/folders/3r/fkfjb6j92yl5dmn63k5ggz4m0000gn/T/ipykernel_88531/4239639834.py:171: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) AppleGothic.
  plt.savefig(save_path, dpi=150, bbox_inches='tight')


   시간대별 SHAP 분석 중...
   ✅ JUNGLE 분석 완료 -> output/shap_analysis/JUNGLE

📂 [MIDDLE] 데이터 및 모델 로딩 중...
   데이터 크기: 2,177,986행
   ✅ 모델 로드 완료: xgb_gold_predictor_MIDDLE_20251207_171358.pkl
   SHAP 값 계산 시작 (샘플: 5,000행)...
   ✅ SHAP 계산 완료
   🔥 Top 5 Features: ['xp', 'minionsKilled', 'kills', 'totalDamageDoneToChampions', 'turretPlates']


...

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>